### Import Libraries

In [2]:
from num2words import num2words
import csv

In [37]:
import pandas as pd
import numpy as np

### Create Database

In [3]:
start = 0
end = 299

In [4]:
filename = "number_to_words_dataset.csv"


In [5]:
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["input", "target"])
    
    for i in range(start, end + 1):
        number = str(i)
        words = num2words(i).replace('-', ' ').replace(',', '')  
        writer.writerow([number, words.lower()])

In [7]:
df = pd.read_csv('number_to_words_dataset.csv')
print(df.head())

   input target
0      0   zero
1      1    one
2      2    two
3      3  three
4      4   four


In [8]:
inputs = df['input'].astype(str).tolist()

In [9]:
targets = df['target'].astype(str).str.lower().tolist()

In [10]:
inputs

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [11]:
targets

['zero',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'twenty',
 'twenty one',
 'twenty two',
 'twenty three',
 'twenty four',
 'twenty five',
 'twenty six',
 'twenty seven',
 'twenty eight',
 'twenty nine',
 'thirty',
 'thirty one',
 'thirty two',
 'thirty three',
 'thirty four',
 'thirty five',
 'thirty six',
 'thirty seven',
 'thirty eight',
 'thirty nine',
 'forty',
 'forty one',
 'forty two',
 'forty three',
 'forty four',
 'forty five',
 'forty six',
 'forty seven',
 'forty eight',
 'forty nine',
 'fifty',
 'fifty one',
 'fifty two',
 'fifty three',
 'fifty four',
 'fifty five',
 'fifty six',
 'fifty seven',
 'fifty eight',
 'fifty nine',
 'sixty',
 'sixty one',
 'sixty two',
 'sixty three',
 'sixty four',
 'sixty five',
 'sixty six',
 'sixty seven',
 'sixty eight',
 'sixty nine',
 'seventy',
 'seventy one',
 'seventy two',
 

In [12]:
targets_input = ['<sos> ' + t for t in targets]
targets_output = [t + ' <eos>' for t in targets]

### Text Preprocessing

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Text Tokenization

In [14]:
input_tokenizer = Tokenizer(char_level=True, filters='')
input_tokenizer.fit_on_texts(inputs)
encoder_input_seq = input_tokenizer.texts_to_sequences(inputs)

In [38]:
target_tokenizer = Tokenizer(char_level=False, lower=True, filters='')
target_tokenizer.fit_on_texts(targets_input + targets_output)
decoder_input_seq = target_tokenizer.texts_to_sequences(targets_input)
decoder_target_seq = target_tokenizer.texts_to_sequences(targets_output)

### Padding

In [44]:
max_encoder_seq_length = max(len(seq) for seq in encoder_input_seq)
max_decoder_seq_length = max(len(seq) for seq in decoder_input_seq)

In [45]:
encoder_input_seq = pad_sequences(encoder_input_seq, maxlen=max_encoder_seq_length, padding='post')
decoder_input_seq = pad_sequences(decoder_input_seq, maxlen=max_decoder_seq_length, padding='post')
decoder_target_seq = pad_sequences(decoder_target_seq, maxlen=max_decoder_seq_length, padding='post')

In [46]:
print("Encoder input shape:", encoder_input_seq.shape)
print("Decoder input shape:", decoder_input_seq.shape)
print("Decoder target shape:", decoder_target_seq.shape)

Encoder input shape: (300, 3)
Decoder input shape: (300, 6)
Decoder target shape: (300, 6)


In [70]:
print("Encoder input dtype:", encoder_input_seq.dtype)
print("Decoder input dtype:", decoder_input_seq.dtype)
print("Decoder target dtype:", decoder_target_seq.dtype)

Encoder input dtype: int32
Decoder input dtype: int32
Decoder target dtype: int32


In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [48]:
encoder_vocab_size = len(input_tokenizer.word_index) + 1
decoder_vocab_size = len(target_tokenizer.word_index) + 1

In [49]:
embedding_dim = 64
lstm_units = 128

### Model Architecture

In [98]:
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=encoder_vocab_size, output_dim=embedding_dim, name='encoder_embedding')(encoder_inputs)
encoder_lstm1, state_h1, state_c1 = LSTM(lstm_units, return_sequences=True, return_state=True, name='encoder_lstm1')(encoder_embedding)
encoder_lstm2, state_h2, state_c2 = LSTM(lstm_units, return_state=True, name='encoder_lstm2')(encoder_lstm1)
encoder_states = [state_h, state_c]

In [99]:
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=decoder_vocab_size, output_dim=embedding_dim, name='decoder_embedding')(decoder_inputs)
decoder_lstm1 = LSTM(lstm_units, return_sequences=True, return_state=True, name='decoder_lstm1')
decoder_output1, _, _ = decoder_lstm1(decoder_embedding, initial_state=[state_h2, state_c2])

# Second LSTM builds deeper context
decoder_lstm2 = LSTM(lstm_units, return_sequences=True, return_state=True, name='decoder_lstm2')
decoder_outputs, _, _ = decoder_lstm2(decoder_output1)

# Final Dense layer
decoder_dense = Dense(decoder_vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)


In [100]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [101]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 encoder_embedding (Embeddi  (None, None, 64)             704       ['encoder_inputs[0][0]']      
 ng)                                                                                              
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                         

In [102]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [103]:
decoder_target_seq = np.expand_dims(decoder_target_seq, -1)

In [75]:
print("Encoder shape:", encoder_input_seq.shape)
print("Decoder input shape:", decoder_input_seq.shape)
print("Decoder target shape:", decoder_target_seq.shape)

Encoder shape: (300, 3)
Decoder input shape: (300, 6)
Decoder target shape: (300, 6, 1, 1, 1)


### Model Training

In [105]:
model.fit(
    [encoder_input_seq, decoder_input_seq],
    decoder_target_seq,
    batch_size=64,
    epochs=150,
    validation_split=0.2
)


Epoch 1/150
4/4 [==============================] - 0s 45ms/step - loss: 0.4902 - accuracy: 0.1306 - val_loss: 0.7735 - val_accuracy: 0.1205
Epoch 2/150
4/4 [==============================] - 0s 30ms/step - loss: 0.4787 - accuracy: 0.1327 - val_loss: 0.7645 - val_accuracy: 0.1197
Epoch 3/150
4/4 [==============================] - 0s 33ms/step - loss: 0.4772 - accuracy: 0.1332 - val_loss: 0.7637 - val_accuracy: 0.1220
Epoch 4/150
4/4 [==============================] - 0s 35ms/step - loss: 0.4716 - accuracy: 0.1300 - val_loss: 0.7453 - val_accuracy: 0.1188
Epoch 5/150
4/4 [==============================] - 0s 33ms/step - loss: 0.4638 - accuracy: 0.1310 - val_loss: 0.7376 - val_accuracy: 0.1166
Epoch 6/150
4/4 [==============================] - 0s 33ms/step - loss: 0.4541 - accuracy: 0.1335 - val_loss: 0.7755 - val_accuracy: 0.1196
Epoch 7/150
4/4 [==============================] - 0s 51ms/step - loss: 0.4513 - accuracy: 0.1299 - val_loss: 0.7685 - val_accuracy: 0.1192
Epoch 8/150
4/4 [===

4/4 [==============================] - 0s 54ms/step - loss: 0.1186 - accuracy: 0.1302 - val_loss: 0.5554 - val_accuracy: 0.1211
Epoch 118/150
4/4 [==============================] - 0s 42ms/step - loss: 0.1183 - accuracy: 0.1314 - val_loss: 0.5466 - val_accuracy: 0.1214
Epoch 119/150
4/4 [==============================] - 0s 49ms/step - loss: 0.1176 - accuracy: 0.1309 - val_loss: 0.5426 - val_accuracy: 0.1208
Epoch 120/150
4/4 [==============================] - 0s 59ms/step - loss: 0.1200 - accuracy: 0.1312 - val_loss: 0.5722 - val_accuracy: 0.1210
Epoch 121/150
4/4 [==============================] - 0s 57ms/step - loss: 0.1181 - accuracy: 0.1295 - val_loss: 0.5892 - val_accuracy: 0.1203
Epoch 122/150
4/4 [==============================] - 0s 50ms/step - loss: 0.1187 - accuracy: 0.1295 - val_loss: 0.5580 - val_accuracy: 0.1213
Epoch 123/150
4/4 [==============================] - 0s 57ms/step - loss: 0.1134 - accuracy: 0.1302 - val_loss: 0.5301 - val_accuracy: 0.1206
Epoch 124/150
4/4 [=

In [107]:
encoder_inputs = model.input[0]  # same as Input(shape=(None,))
encoder_lstm1 = model.get_layer('encoder_lstm1')
encoder_lstm2 = model.get_layer('encoder_lstm2')

# Apply stacked LSTM layers
encoder_output1, _, _ = encoder_lstm1(encoder_embedding)
_, state_h, state_c = encoder_lstm2(encoder_output1)
encoder_states = [state_h, state_c]


In [108]:
encoder_model = Model(encoder_inputs, encoder_states)

### Make Predictions

In [119]:
decoder_inputs = Input(shape=(None,), name='decoder_inputs_infer')

# Decoder state inputs (for 2 LSTM layers)
decoder_state_input_h1 = Input(shape=(lstm_units,), name='decoder_h1')
decoder_state_input_c1 = Input(shape=(lstm_units,), name='decoder_c1')
decoder_state_input_h2 = Input(shape=(lstm_units,), name='decoder_h2')
decoder_state_input_c2 = Input(shape=(lstm_units,), name='decoder_c2')
decoder_states_inputs = [decoder_state_input_h1, decoder_state_input_c1,
                         decoder_state_input_h2, decoder_state_input_c2]

# Get the layers from the trained model
decoder_embedding_layer = model.get_layer('decoder_embedding')
decoder_lstm1 = model.get_layer('decoder_lstm1')
decoder_lstm2 = model.get_layer('decoder_lstm2')
decoder_dense = model.get_layer('decoder_dense')

# Reuse the layers with new inputs
embedded_inputs = decoder_embedding_layer(decoder_inputs)

# First LSTM layer
decoder_output1, state_h1, state_c1 = decoder_lstm1(
    embedded_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1]
)

# Second LSTM layer
decoder_output2, state_h2, state_c2 = decoder_lstm2(
    decoder_output1, initial_state=[decoder_state_input_h2, decoder_state_input_c2]
)

# Dense layer to get probabilities
decoder_outputs = decoder_dense(decoder_output2)

In [120]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs, state_h1, state_c1, state_h2, state_c2]
)

In [129]:
reverse_target_index = {i: word for word, i in target_tokenizer.word_index.items()}
sos_token = target_tokenizer.word_index['<sos>']
eos_token = target_tokenizer.word_index['<eos>']


In [ ]:
def number_to_words(number_str):
    # Preprocess input number string
    input_seq = input_tokenizer.texts_to_sequences([number_str])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')

    # Encode input to get initial states
    states_value = encoder_model.predict(input_seq)

    # Unpack encoder states
    state_h1, state_c1, state_h2, state_c2 = states_value[0], states_value[1], states_value[0], states_value[1]

    # Prepare initial decoder input (<sos>)
    target_seq = np.array([[sos_token]])

    decoded_sentence = ''
    stop_condition = False

    while not stop_condition:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict(
            [target_seq, state_h1, state_c1, state_h2, state_c2]
        )

        # Get most likely token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_index.get(sampled_token_index, '')

        if sampled_word == '<eos>' or len(decoded_sentence.split()) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Update target_seq and states for next timestep
        target_seq = np.array([[sampled_token_index]])
        state_h1, state_c1, state_h2, state_c2 = h1, c1, h2, c2

    return decoded_sentence.strip()


In [138]:
print(number_to_words("50"))

1/1 [==============================] - 0s 35ms/step
fifty
